In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

In [2]:
from autogen import ConversableAgent

/home/codespace/.local/lib/python3.12/site-packages/flaml/__init__.py:20: UserWarning: flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.
  warnings.warn("flaml.automl is not available. Please install flaml[automl] to enable AutoML functionalities.")


In [3]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

[autogen.oai.client: 03-04 04:57:54] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [4]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

[autogen.oai.client: 03-04 04:57:54] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [5]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

[autogen.oai.client: 03-04 04:57:54] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


In [6]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

## Creating series of tasks

In [7]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## Start the onboarding process¶

In [8]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


/home/codespace/.local/lib/python3.12/site-packages/autogen/agentchat/chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


customer_proxy_agent (to Onboarding Personal Information Agent):

sohail

--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Thank you for providing your name. Could you also let me know your location, please?

--------------------------------------------------------------------------------
customer_proxy_agent (to Onboarding Personal Information Agent):

putin

--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Topic preference Agent (to customer_proxy_agent):

Great! Could you tell me what topics you are interested in reading about?
Context: 
{'name': 'sohail', 'location': 'putin'}

--------------------------------------------------------------------------------
cu

In [9]:
chat_results

[ChatResult(chat_id=None, chat_history=[{'content': "Hello, I'm here to help you get started with our product.Could you tell me your name and location?", 'role': 'assistant'}, {'content': 'sohail', 'role': 'user'}, {'content': 'Thank you for providing your name. Could you also let me know your location, please?', 'role': 'assistant'}, {'content': 'putin', 'role': 'user'}], summary="{'name': 'sohail', 'location': 'putin'}", cost={'usage_including_cached_inference': {'total_cost': 0.00014649999999999998, 'gpt-3.5-turbo-0125': {'cost': 0.00014649999999999998, 'prompt_tokens': 188, 'completion_tokens': 35, 'total_tokens': 223}}, 'usage_excluding_cached_inference': {'total_cost': 0.00014649999999999998, 'gpt-3.5-turbo-0125': {'cost': 0.00014649999999999998, 'prompt_tokens': 188, 'completion_tokens': 35, 'total_tokens': 223}}}, human_input=[]),
 ChatResult(chat_id=None, chat_history=[{'content': "Great! Could you tell me what topics you are interested in reading about?\nContext: \n{'name': '

In [10]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{'name': 'sohail', 'location': 'putin'}


sohail from putin is interested in reading about trump.


Sohail from Putin is interested in reading about Donald Trump, who was a reality TV star before becoming the President of the United States.




In [11]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.00014649999999999998, 'gpt-3.5-turbo-0125': {'cost': 0.00014649999999999998, 'prompt_tokens': 188, 'completion_tokens': 35, 'total_tokens': 223}}, 'usage_excluding_cached_inference': {'total_cost': 0.00014649999999999998, 'gpt-3.5-turbo-0125': {'cost': 0.00014649999999999998, 'prompt_tokens': 188, 'completion_tokens': 35, 'total_tokens': 223}}}


{'usage_including_cached_inference': {'total_cost': 5.4000000000000005e-05, 'gpt-3.5-turbo-0125': {'cost': 5.4000000000000005e-05, 'prompt_tokens': 66, 'completion_tokens': 14, 'total_tokens': 80}}, 'usage_excluding_cached_inference': {'total_cost': 5.4000000000000005e-05, 'gpt-3.5-turbo-0125': {'cost': 5.4000000000000005e-05, 'prompt_tokens': 66, 'completion_tokens': 14, 'total_tokens': 80}}}


{'usage_including_cached_inference': {'total_cost': 0.000299, 'gpt-3.5-turbo-0125': {'cost': 0.000299, 'prompt_tokens': 265, 'completion_tokens': 111, 'total_tokens': 376}}, 'usage_excluding_cached_